In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading data

In [2]:
df = pd.read_csv("../input/alpha123/trainingdata.csv", delimiter='\t', encoding='latin-1', header=None)

In [3]:
df[0]

# Data Cleaning

In [4]:
tagging = []
question = []

In [5]:
for _, row in df.iterrows():
    x = row[0]
    a = ""
    b = ""
    for i in x:
        if(i==" "):
            break
        else:
            a = a+i
    b = x[len(a)+1:]
    tagging.append(a)
    question.append(b)
            

In [6]:
tagging = pd.Series(tagging)
question = pd.Series(question)

In [7]:
data = {"tagging":tagging, "question":question}

In [8]:
df = pd.concat(data, axis = 1)

# Printing Cleaned Data

In [9]:
df.head()

In [10]:
df.groupby([tagging]).count()

# Preprocessing Data

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['tagging'] = le.fit_transform(df['tagging'])

In [12]:
df['tagging']

# Creating training and testing set

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['question'], 
                                                    df['tagging'], 
                                                    random_state=42,test_size=0.20)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
X_train = count_vector.fit_transform(X_train)
X_test = count_vector.transform(X_test)

# Training Naive Bayes Classifier

In [16]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha = 0.5, fit_prior = 'True')
model.fit(X_train, y_train)

# Testing Model

In [23]:
from sklearn.metrics import accuracy_score
predictions_test = model.predict(X_test)
accuracy = accuracy_score(predictions_test,y_test)
print(accuracy)

In [29]:
def makePrediction(ques):
    ques = np.array([ques])
    ques = count_vector.transform(ques)
    pred = model.predict(ques)
    print(le.inverse_transform(pred))

In [30]:
ques = "How did serfdom develop in and then leave Russia ?"
makePrediction(ques)

# Saving Model

In [31]:
import pickle
pickle_out = open("vectorizer.pkl","wb")
pickle.dump(count_vector, pickle_out)
pickle_out.close()

In [ ]:
import pickle
pickle_out = open("label.pkl","wb")
pickle.dump(le, pickle_out)
pickle_out.close()

In [33]:
pickle_out = open("NaiveBayes.pkl","wb")
pickle.dump(model, pickle_out)
pickle_out.close()